In [1]:
from itertools import permutations
from copy import deepcopy

def testBjct(bjct, a, b):
    for j in a:
        for k in a:
            # translate to other's elements
            t_j, t_k = bjct[j], bjct[k]
            # check if bijection gives the right value
            if bjct[a[j][k]] != b[t_j][t_k]:
                return False
    return True

class Group:
    def __init__(self, elm, data = {}, op = print):
        # this dictionary stores all elements of the group table
        self.data    = {}
        # stores the order of each element and a list which holds generators (also elements keyed on order)
        self.orders  = {}
        self.ord_elm = {}
        self.gens    = []
        # a set which contains the elements
        self.elm     = set(elm)
        # the identity element
        self.id      = None
        # boolean which indicates abelian and/or cyclic
        self.abelian = None
        self.cyclic  = None
        
        # Verify we have some input to go off of
        if len(data) != len(elm):
            if op == print:
                raise Exception("Neither data nor func are well-defined")
            # function was passed in
            else:
                # make table from func
                for i in elm:
                    self.data[i] = {}
                    for j in elm: 
                        self.data[i][j] = op(i, j)
        else:
            # data was passed in
            # check that we have appropriate parameters
            for i in elm:
                # check that key exists
                if i not in data:
                    raise Exception("missing key " + str(i))
                # check that subkey exists
                for j in data:
                    if i not in data[j]:
                        raise Exception("missing key " + str(j) + " from " + str(i))
            # we're good to go! save data
            self.data = data
        
        # verify that this is actually a group
        if not self.verify(verbose=True):
            print("Warning! Not a group! Further behavior may be undefined.")
    
    def __repr__(self):
        # sort our elements to be fancy (and stable)
        elm = sorted(list(self.elm))
        
        # find width size
        width = max([len(str(i)) for i in elm]) + 1
        out = " " * width + " |"
        
        # first row labels
        for i in elm:
            out += "{0:>{2}}{1:>2}".format(str(i), "|", width)
        out += " Order"
        row_width = len(out) + 1
        out += "\n" + "-" * row_width
        
        for i in elm:
            # print row header
            out += "\n{0:>{2}} {1:}".format(str(i), "|", width)
            # print elements
            for j in elm:
                out += "{0:>{2}}{1:>2}".format(str(self.data[i][j]), "|", width)
            out += " {}".format(self.orders[i])
            out += "\n" + "-" * row_width
        return out
    
    def __eq__(self, other):
        # do some basic checks
        # check if both groups have same n elements
        if (len(self.elm) != len(other.elm)):
            return False
        # check if group has some n of elements of i order
        for i in self.ord_elm:
            if len(self.ord_elm[i]) != len(other.ord_elm[i]):
                return False
        
        # generate possible bijections
        p_bjct = [{}]
        # for each unqiue order
        for i in self.ord_elm:
            # make a new list to store new bijections
            new_bjct = []
            # for every possible combination of elements with this order
            for j in permutations(self.ord_elm[i]):
                # for every existing bijection
                for k in p_bjct:
                    # make a copy of the bijection
                    bjct = deepcopy(k)
                    # and add the new mapping to this bijection
                    for h in range(len(j)):
                        bjct[j[h]] = other.ord_elm[i][h]
                    # add bijection to running list
                    new_bjct += [bjct]
            # replace possible bijections with newly updated bijections
            p_bjct = new_bjct
        
        # test the bijections we generated
        for i in p_bjct:
            if testBjct(i, self.data, other.data):
                print(i)
                return True
            
        return False
    
    def __ne__(self, other):
        return not self.__eq__(other)
    
    def verify(self, verbose=True):
        # default group values
        self.cyclic  = False
        self.abelian = True
        self.gens    = []
        
        # find identity element, check closure, verify associativity
        for i in self.elm:
            # determine if elegible for id elmnt
            id_cand = False
            if self.data[i][i] == i:
                id_cand = True
            
            for j in self.elm:
                # if id check, make sure that id is returned
                # otherwise, just check closure
                if id_cand:
                    if self.data[i][j] != j:
                        if verbose:
                            print(str(i) + " + " + str(j) + " != " + str(j))
                        return False
                else:
                    if self.data[i][j] not in self.elm:
                        print(str(self.data[i][j]) + " is not in " + str(self.elm))
                        return False
                
                # verify associativity
                # h + (i + j) = (h + i) + j
                for h in self.elm:
                    if self.data[h][self.data[i][j]] != self.data[self.data[h][i]][j]:
                        if verbose:
                            print(str(h) + " + (" + str(i) + " + " + str(j) + ")"
                             + " != " + " (" + str(h) + " + " + str(i) + ")")
                        return False
                
                # Just for fun, let's also verify commutativity
                if self.data[i][j] != self.data[j][i]:
                    self.abelian = False
                
            # check id
            if id_cand:
                if verbose: print("id element found: " + str(i))
                self.id = i
                id_cand = False
                
        if verbose:
            print("Associative           ✓")
            print("Closed under operator ✓")

        # check that an ID element was found
        if self.id == None:
            if verbose: print("No identity element found!")
            return False
        else:
            if verbose: print("Identity element      ✓")

        
        # Check for inversions and find order
        for i in self.elm:
            found = False
            for j in self.elm:
                if self.data[i][j] == self.id:
                    found = True
            if not found:
                if verbose: print(str(i) + " has no inversion!")
                return False
            # this is the element which we are operating on
            _i = i
            order = 1
            # keep operating until we get id
            while _i != self.id:
                _i = self.data[i][_i]
                order += 1
                if order > len(self.elm):
                    order = float("inf")
                    break
            # add to dictionary and detect generators
            self.orders[i]      = order
            if (order not in self.ord_elm):
                self.ord_elm[order] = []
            self.ord_elm[order] += [i]
            
            if order == len(self.elm):
                self.gens  += [i]
                self.cyclic = True
        
        # We've checked everything!
        if verbose:
            print("Inversions exist      ✓")
            print("Is group              ✓")
            if self.abelian:
                print("Abelian               ✓")
            else:
                print("Abelian               ✘")
            if self.cyclic:
                print("Cyclic                ✓")
            else:
                print("Cyclic                ✘")
                
        return True

#def tup(a, b):
#    return (a[0]*b[0],a[1]*b[1])

#b = Group([(1,1),(1,-1),(-1,1),(-1,-1)], op=tup)
#print(b)
#print("=" * 30)
def mod5(a, b):
    return (a * b) % 5
def mod10(a, b):
    return (a * b) % 10
#def mod229(a, b):
#    return (a * b) % 229
#from time import time
#before = time()
a = Group([*range(1,5)], op=mod5)
print(a)

b = Group([1,3,7,9], op=mod10)
print(b)

assert a == b
#print(time() - before)
#print()

#prime_list = [2] + [*filter(lambda i:all(i%j for j in range(3,i,2)), range(3,10000,2))] 
#print(prime_list)

id element found: 1
Associative           ✓
Closed under operator ✓
Identity element      ✓
Inversions exist      ✓
Is group              ✓
Abelian               ✓
Cyclic                ✓
   | 1 | 2 | 3 | 4 | Order
---------------------------
 1 | 1 | 2 | 3 | 4 | 1
---------------------------
 2 | 2 | 4 | 1 | 3 | 4
---------------------------
 3 | 3 | 1 | 4 | 2 | 4
---------------------------
 4 | 4 | 3 | 2 | 1 | 2
---------------------------
id element found: 1
Associative           ✓
Closed under operator ✓
Identity element      ✓
Inversions exist      ✓
Is group              ✓
Abelian               ✓
Cyclic                ✓
   | 1 | 3 | 7 | 9 | Order
---------------------------
 1 | 1 | 3 | 7 | 9 | 1
---------------------------
 3 | 3 | 9 | 1 | 7 | 4
---------------------------
 7 | 7 | 1 | 9 | 3 | 4
---------------------------
 9 | 9 | 7 | 3 | 1 | 2
---------------------------
{1: 1, 2: 3, 3: 7, 4: 9}


## 2.10

#### 2) Let $G$ have as elements the four pairs $(1,1),(1,-1),(-1,1),(-1,-1)$, and let $(a,b)\oplus(c,d)=(ac,bd)$. Prove that $G$ is a group.

In [2]:
def g1_op(a, b):
    return (a[0] * b[0], a[1] * b[1])

g_1 = Group([(1,1),(1,-1),(-1,1),(-1,-1)], op=g1_op)
print(g_1)

id element found: (1, 1)
Associative           ✓
Closed under operator ✓
Identity element      ✓
Inversions exist      ✓
Is group              ✓
Abelian               ✓
Cyclic                ✘
          | (-1, -1) |  (-1, 1) |  (1, -1) |   (1, 1) | Order
--------------------------------------------------------------
 (-1, -1) |   (1, 1) |  (1, -1) |  (-1, 1) | (-1, -1) | 2
--------------------------------------------------------------
  (-1, 1) |  (1, -1) |   (1, 1) | (-1, -1) |  (-1, 1) | 2
--------------------------------------------------------------
  (1, -1) |  (-1, 1) | (-1, -1) |   (1, 1) |  (1, -1) | 2
--------------------------------------------------------------
   (1, 1) | (-1, -1) |  (-1, 1) |  (1, -1) |   (1, 1) | 1
--------------------------------------------------------------


#### 4) Prove that the set of elements $e, a, b, c$ with the following table for the binary operation is a group. Prove that this group is isomorphic to the additive group modulo 4.

In [3]:
print("Information for group with elements e, a, b, c")
g_2 = Group(["e", "a", "b", "c"], {
    "e": {"e": "e", "a": "a", "b": "b", "c": "c"},
    "a": {"e": "a", "a": "e", "b": "c", "c": "b"},
    "b": {"e": "b", "a": "c", "b": "a", "c": "e"},
    "c": {"e": "c", "a": "b", "b": "e", "c": "a"}
})
print(g_2)

print("\nInformation for additive group under mod 4")
def add4(a, b):
    return (a + b) % 4

g_3 = Group([0, 1, 2, 3], op=add4)
print(g_3)

print("\nAttempting to find bijection between two groups...")
if g_2 == g_3:
    print("Bijection found! Groups are isomorphic. ✓")
else:
    print("Math machine broke.")

Information for group with elements e, a, b, c
id element found: e
Associative           ✓
Closed under operator ✓
Identity element      ✓
Inversions exist      ✓
Is group              ✓
Abelian               ✓
Cyclic                ✓
   | a | b | c | e | Order
---------------------------
 a | e | c | b | a | 2
---------------------------
 b | c | a | e | b | 4
---------------------------
 c | b | e | a | c | 4
---------------------------
 e | a | b | c | e | 1
---------------------------

Information for additive group under mod 4
id element found: 0
Associative           ✓
Closed under operator ✓
Identity element      ✓
Inversions exist      ✓
Is group              ✓
Abelian               ✓
Cyclic                ✓
   | 0 | 1 | 2 | 3 | Order
---------------------------
 0 | 0 | 1 | 2 | 3 | 1
---------------------------
 1 | 1 | 2 | 3 | 0 | 4
---------------------------
 2 | 2 | 3 | 0 | 1 | 2
---------------------------
 3 | 3 | 0 | 1 | 2 | 4
---------------------------

Attempting to 

#### 5)

#### Prove that the set of elements $e, u, v, w,$ with the following table for the binary operation is a group. Prove that this group is not isomorphic to the additive group modulo 4, but that it is isomorphic to the group described in Problem 2.

In [14]:
print("Creating and verifying Group g_4:")
g_4 = Group(["e", "u", "v", "w"], {
    "e": {"e": "e", "u": "u", "v": "v", "w": "w"},
    "u": {"e": "u", "u": "e", "v": "w", "w": "v"},
    "v": {"e": "v", "u": "w", "v": "e", "w": "u"},
    "w": {"e": "w", "u": "v", "v": "u", "w": "e"}
})
print(str(g_4))

print("\n")

if g_4 == g_3:
    print("Math machine broke")
else:
    print("Additive group mod 4 not isomorphic to g_4\n")
    
if g_4 == g_1:
    print("g_4 is isomorphic to g_1 ✓")
else:
    print("Math machine broke")

Creating and verifying Group g_4:
id element found: e
Associative           ✓
Closed under operator ✓
Identity element      ✓
Inversions exist      ✓
Is group              ✓
Abelian               ✓
Cyclic                ✘
   | e | u | v | w | Order
---------------------------
 e | e | u | v | w | 1
---------------------------
 u | u | e | w | v | 2
---------------------------
 v | v | w | e | u | 2
---------------------------
 w | w | v | u | e | 2
---------------------------


Additive group mod 4 not isomorphic to g_4

{'e': (1, 1), 'v': (-1, 1), 'w': (-1, -1), 'u': (1, -1)}
g_4 is isomorphic to g_1 ✓


#### 6)

In [20]:
def mod5(a, b):
    return (a * b) % 5

print("Creating and verifying Group g_5 under mod 5")
g_5 = Group([1,2,3,4], op=mod5)
print(g_5)

print("\nChecking if isomorphic to g_3")
if (g_3 == g_5):
    print("Bijection found!")
else:
    print("Math machine broke")

Creating and verifying Group g_5 under mod 5
id element found: 1
Associative           ✓
Closed under operator ✓
Identity element      ✓
Inversions exist      ✓
Is group              ✓
Abelian               ✓
Cyclic                ✓
   | 1 | 2 | 3 | 4 | Order
---------------------------
 1 | 1 | 2 | 3 | 4 | 1
---------------------------
 2 | 2 | 4 | 1 | 3 | 4
---------------------------
 3 | 3 | 1 | 4 | 2 | 4
---------------------------
 4 | 4 | 3 | 2 | 1 | 2
---------------------------

Checking if isomorphic to g_3
{0: 1, 1: 2, 3: 3, 2: 4}
Bijection found!


## 2.1

#### 23) Prove that $n^{13} - n$ is divisble by $2,3,5,7$ and $13$ for any integer $n$.



Example:

Show that $x^{19}-x$ is divisible by $7$.

We'll work in the group $\frac{\mathbb{Z}}{7\mathbb{Z}}$.

First, if $[x]=[0]$, then $[x^2-x]=[0]^2-[0]=[0]$, so $x$ is divisible by $7$.

Let $[x] \neq [0]$. Then $[x] \in (\frac{\mathbb{Z}}{7\mathbb{Z}})^*$ whose order is $7-1=6$. Therefore $[x]^6=[1]$.

Since $x^{19}-x=x(x^{18}-1)$, it's sufficient to show that $7|x^{18}-1$, or equivalently, $[x]^{18}=[1]$

But $[x]^{18}=[x^3]^6=[1]$.

Example:

Show that $(\frac{\mathbb{Z}}{10\mathbb{Z}})^*$ is ismorphic to $(\frac{\mathbb{Z}}{5\mathbb{Z}})^*$.

In [5]:
## Euclidean Algorithm
def euclid(a, b):
    """Finds the gcd between two numbers"""
    a, b = sorted((a, b))
    
    # y = coeff(x) + rem
    # repeat until remainder is 0
    rem = -1
    coeff = 0
    
    while(rem != 0):
        coeff = a // b
        rem =   a - coeff * b
        a = b
        b = rem
    return a

In [6]:
def mod_group(n):
    """Creates group (Z/nZ)*"""
    # generate a list of coprimes < n
    coprime = [i for i in range(1,n) if euclid(i, n) == 1]
    out = []
    for i in coprime:
        row = []
        for j in coprime:
            row += [(i * j) % n]
        out += [row]
    return out, coprime

mod_group(5)

([[1, 2, 3, 4], [2, 4, 1, 3], [3, 1, 4, 2], [4, 3, 2, 1]], [1, 2, 3, 4])

In [7]:
class _Group:
    def __init__(self, labels, data = ):
        assert len(labels) == len(data)
        
        self.data = data
        self.labels = labels
        
    def __repr__(self):
        out = ""
        # first row labels
        for i in [" "] + self.labels:
            out += "{:3}".format(i)
        
        for i in range(len(self.data)):
            out += "\n\n{:3}".format(self.labels[i])
            for j in self.data[i]:
                out += "{:3}".format(j)
        return out
    
    def __eq__(self, other):
        if set(self.labels) != set(other.labels):
            return False
        
        # generate translation tables
        o_trans = {}
        for i in other.labels:
            o_trans[i] = other.labels.index(i)
        
        # compare translated values
        for i in range(len(self.data)):
            for j in range(len(self.data[i])):
                if self.data[i][j] != other.data[o_trans[self.labels[i]]][o_trans[self.labels[j]]]:
                    return False
        return True
    
    def __ne__(self, other):
        return not self == other
    
    def apply_bijection(self, bijection):
        for i in range(len(self.data)):
            for j in range(len(self.data[i])):
                self.data[i][j] = bijection[self.data[i][j]]
        for i in range(len(self.labels)):
            self.labels[i] = bijection[self.labels[i]]
    

SyntaxError: invalid syntax (<ipython-input-7-5a61b198e7a9>, line 2)

In [ ]:
from itertools import permutations

# Generates possible bijections between two sets of group elements
def gen_biject(s_1: list, s_2: list) -> list:
    # make sure we have compatable inputs
    assert len(s_1) == len(s_2)
    
    # initialize output and generate permutations
    out = []
    possible = [i for i in permutations(s_1)]
    
    # make dictionaries from the permutations
    for i in range(len(possible)):
        translate = dict()
        for j in range(len(possible[i])):
            translate[s_2[j]] = possible[i][j]
        out += [translate]
    
    # return list of bijection permutations
    return out
gen_biject([1,2,3,4], [1,2,3,4])

In [ ]:
from copy import deepcopy

a = Group([[0,1,2,3],[1,2,3,0],[2,3,0,1],[3,0,1,2]],[0,1,2,3])
Z_5 = Group(*mod_group(5))
Z_10 = Group(*mod_group(10))


for i in gen_biject(Z_5.labels, a.labels):
    tmp = deepcopy(a)
    tmp.apply_bijection(i)
    if (tmp == Z_5):
        print(i)
        break

In [ ]:
abby = Group(*mod_group(18))
Z_7 = Group(*mod_group(7))

for i in gen_biject(abby.labels, Z_7.labels):
    tmp = deepcopy(Z_7)
    tmp.apply_bijection(i)
    if (tmp == abby):
        print(i)
        break